<a href="https://colab.research.google.com/github/AgomferAustral/DMA-Caras/blob/main/EigenfacesMZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# conexion al Google Drive
from google.colab import drive
drive.mount('/content/.drive')

Drive already mounted at /content/.drive; to attempt to forcibly remount, call drive.mount("/content/.drive", force_remount=True).


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Importacion de librerias
import mediapipe as mp

In [4]:
import os
import cv2
import sys
import numpy as np

In [ ]:
!pip install --upgrade numpy
!pip install --upgrade scipy

In [ ]:
!pip install --upgrade mediapipe

In [6]:
# Definicion de ruta de conexion
ruta_imagenes = '/content/.drive/My Drive/Eigenfaces'

In [8]:
def procesar_archivos_en_carpetas(ruta_principal):
  """Recorre las carpetas dentro de la ruta principal y procesa los archivos.

  Args:
    ruta_principal: La ruta de la carpeta principal.
  """
  print(f"Ruta principal: {ruta_principal}")
  for carpeta_actual, _, archivos in os.walk(ruta_principal):
    print(f"Carpeta actual: {carpeta_actual}")
    for archivo in archivos:
      print(f"Archivo: {archivo}")
      ruta_completa = os.path.join(carpeta_actual, archivo)
      try:
        # Aquí va el código para procesar el archivo
        # Por ejemplo, leer el archivo y realizar alguna operación
        with open(ruta_completa, 'r') as f:
          contenido = f.read()
          print(f"Contenido del archivo {ruta_completa}:\n{contenido}")

      except Exception as e:
        print(f"Error al procesar el archivo {ruta_completa}: {e}")

# Ejemplo de uso:
print("Procesando archivos")
procesar_archivos_en_carpetas(ruta_imagenes)
print("Proceso inicial terminado ...")

Procesando archivos
Proceso inicial terminado ...


In [ ]:
# Carga de imagenes
imagenes = []
for nombre_archivo in os.listdir(ruta_imagenes):
       ruta_imagen = os.path.join(ruta_imagenes, nombre_archivo)
       imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)  # Carga en escala de grises
       imagenes.append(imagen)

In [ ]:
print(f"Se cargaron {len(imagenes)} imágenes.")

In [ ]:
# Mostrar imagenes importadas
import matplotlib.pyplot as plt

# Mostrar las primeras 5 imágenes
for i in range(min(5, len(imagenes))):
    plt.imshow(imagenes[i], cmap='gray')
    plt.title(f"Imagen {i + 1}")
    plt.show()

In [ ]:
# Deteccion de rostro
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

In [ ]:
# Recortar imagenes
imagenes_recortadas = []
for imagen in imagenes:
  # Convertir la imagen de escala de grises a RGB
  imagen_rgb = cv2.cvtColor(imagen, cv2.COLOR_GRAY2RGB)

  # Detectar rostros en la imagen
  results = face_detection.process(imagen_rgb)

  # Si se encuentra al menos un rostro
  if results.detections:
    for detection in results.detections:
      # Obtener la bounding box del rostro
      bbox = detection.location_data.relative_bounding_box

      # Calcular las coordenadas de la bounding box en píxeles
      h, w = imagen.shape
      x = int(bbox.xmin * w)
      y = int(bbox.ymin * h)
      width = int(bbox.width * w)
      height = int(bbox.height * h)

      # Recortar la imagen usando la bounding box
      rostro_recortado = imagen[y:y+height, x:x+width]

      # Añadir la imagen recortada a la lista
      imagenes_recortadas.append(rostro_recortado)

face_detection.close()

In [ ]:
import matplotlib.pyplot as plt

# Mostrar las primeras 5 imágenes recortadas
for i in range(min(5, len(imagenes_recortadas))):
    plt.imshow(imagenes_recortadas[i], cmap='gray')
    plt.title(f"Imagen recortada {i + 1}")
    plt.show()

In [ ]:
# Escarlar imagenes
tamaños_comunes = [(70, 70)]

# Escalar las imágenes a cada tamaño común
imagenes_escaladas = [cv2.resize(image, tamaños_comunes[0]) for image in imagenes_recortadas]


In [ ]:
# Visualizar las primeras 10 imágenes escaladas
fig, axes = plt.subplots(2, 5, figsize=(15, 6))  # 2 filas, 5 columnas
fig.suptitle("Primeras 10 Imágenes Escaladas", fontsize=16)

for i, imagen in enumerate(imagenes_escaladas[:10]):
    ax = axes[i // 5, i % 5]  # Calcula la posición en la cuadrícula
    ax.imshow(imagen, cmap='gray')  # Muestra la imagen en escala de grises
    ax.set_title(f"Imagen {i + 1}")

    # Agregar valores en los ejes
    ax.set_xticks(np.arange(0, imagen.shape[1], 10))  # Marcas cada 10 píxeles en el eje x
    ax.set_yticks(np.arange(0, imagen.shape[0], 10))  # Marcas cada 10 píxeles en el eje y
    ax.set_xticklabels(np.arange(0, imagen.shape[1], 10))  # Etiquetas cada 10 píxeles en el eje x
    ax.set_yticklabels(np.arange(0, imagen.shape[0], 10))  # Etiquetas cada 10 píxeles en el eje y

plt.tight_layout()
plt.show()

In [ ]:
# Crecion de matris 2D


# Crear la matriz de datos
m = len(imagenes_escaladas)
d = tamaños_comunes[0][0] * tamaños_comunes[0][1]
X = np.reshape(imagenes_escaladas, (m, d))


In [ ]:
# Obtencion de componentes principales

from sklearn.decomposition import PCA

# Aplicar SVD
U, Sigma, VT = np.linalg.svd(X, full_matrices=False)

# Aplicar PCA usando SVD
pca = PCA(n_components=50)  # Reducir a x componentes principales
pca.fit(X)  # Ajustar PCA a la matriz de datos
componentes_principales = pca.transform(X) # Transformar los datos


#Preprocesamiento de imagenes

##  